In [1]:
# STANDARD LIBRARY IMPORTS
import re
import csv
import os
import datetime

# PROJECT IMPORTS
from setup import reddit
from settings import *

# THIRD PARTY IMPORTS
import praw

Loaded 2 users who have opted out.


In [2]:
subreddit = reddit.subreddit(TARGET_SUBREDDIT)

# flair experiments

In [3]:
awarding_comment = reddit.comment(id='hx3zdnz')
awarded_comment  = reddit.comment(id=awarding_comment.parent_id[3:])
submission       = reddit.submission(id=awarding_comment.link_id[3:])

In [4]:
submission.flair.select(FLAIR_TEMPLATE_ID)

In [5]:
submission.flair.choices()

[{'flair_css_class': '',
  'flair_template_id': '7eeec638-8f07-11ec-8294-2a47d40d3100',
  'flair_text_editable': True,
  'flair_position': 'right',
  'flair_text': 'HPs from op'}]

In [6]:
for f in subreddit.flair(redditor=reddit.redditor('cyclistNerd')): print(f)

{'flair_css_class': None, 'user': Redditor(name='cyclistNerd'), 'flair_text': None}


In [7]:
for f in reddit.subreddit('photocritique').flair(redditor=reddit.redditor('cyclistNerd')): print(f)

{'flair_css_class': 'website', 'user': Redditor(name='cyclistNerd'), 'flair_text': 'galenweld.com'}


In [8]:
reddit.comment(id='hx3zdnz').parent_id[:2]

't1'

In [9]:
datetime.datetime.now().strftime('[%X %x]')

'[18:47:45 02/25/22]'

In [10]:
awarded_comment.id

'hx3z8a5'

# pm testing

In [11]:
SEGMENTOR = re.compile(r'(\n{2,}|\. |\? )')

In [12]:
test_string = '\n\nThis is a reddit comment. It has another sentence.\n\nIt also has a newline\n\nAnd a last\nsentence.'

In [13]:
print(test_string)



This is a reddit comment. It has another sentence.

It also has a newline

And a last
sentence.


In [14]:
sentences = SEGMENTOR.split(test_string)

# the match group gets us the divider, so to keep linebreaks we loop over every even-indexed segment and the one after
segments = []
for i in range(0, len(sentences), 2):
    sentence  = sentences[i]
    separator = sentences[i+1] if i+1<len(sentences) else None
    
    # replace single newlines with spaces, then strip whitespace
    sentence = re.sub('\n', ' ', sentence).strip()
    
    # skip empty sentences
    if len(sentence)==0: continue
        
    # check the separator, if it's two or more newlines, keep it
    if (separator is not None) and (re.match(r'\n{2,}', separator) is not None):
        sentence += '\n\n'
        
    segments.append(sentence)
    
# finally, add quotes and numbers
segments = [f'> ({i+1}) {s}' for i, s in enumerate( segments )] # insert numbers

In [15]:
reply = []
    
reply.append(f'Thanks for indicating that [this comment]({awarded_comment.permalink}) by /u/{awarded_comment.author.name} was helpful.')
reply.append('Please let us know which part of the comment was most helpful.')
reply.append('Reply to this PM  with the number of the sentence(s) you found most useful.')
reply.append('You may reply with multiple numbers, or a range of numbers, e.g. 2-4,6.')
reply.append('\n')
reply += segments

reply = '\n'.join(reply)

print(reply)

Thanks for indicating that [this comment](/r/phcritiquetest/comments/sti3ob/this_is_a_photo_i_took_of_a_pretty_sunrise_i/hx3z8a5/) by /u/CommunityGovProject was helpful.
Please let us know which part of the comment was most helpful.
Reply to this PM  with the number of the sentence(s) you found most useful.
You may reply with multiple numbers, or a range of numbers, e.g. 2-4,6.


> (1) This is a reddit comment
> (2) It has another sentence.


> (3) It also has a newline


> (4) And a last sentence.


In [16]:
# checking PM inbox

In [17]:
reddit.inbox

In [18]:
num_processed = 0

print('Processing unread PMs.')
for message in reddit.inbox.unread(limit=None):
    if not isinstance(message, praw.models.Message): continue
        
    print(message)    
    # message.mark_read()
    num_processed += 1
    
if num_processed == 0:
    print('Finished. No unread PMs.')
else:
    print(f'Finished. Processed {num_processed} PMs.')

Processing unread PMs.
Finished. No unread PMs.


In [19]:
message.author

Redditor(name='uw_cse_moderation')

In [20]:
message.id

'hygqut6'

In [21]:
message.body

"Really great feedback, I'll try booping the snoot. #helpful"

In [22]:
message.created

1645842416.0

In [23]:
vars(message)

{'_replies': [],
 '_submission': None,
 '_reddit': <praw.reddit.Reddit at 0x7f819029a8b0>,
 'first_message': None,
 'first_message_name': None,
 'subreddit': Subreddit(display_name='phcritiquetest'),
 'likes': None,
 'author_fullname': 't2_4nzdo3pe',
 'id': 'hygqut6',
 'subject': 'comment reply',
 'associated_awarding_id': None,
 'score': 1,
 'author': Redditor(name='uw_cse_moderation'),
 'num_comments': 9,
 'parent_id': 't1_hygqs5i',
 'subreddit_name_prefixed': 'r/phcritiquetest',
 'new': True,
 'type': 'comment_reply',
 'body': "Really great feedback, I'll try booping the snoot. #helpful",
 'link_title': 'Mount Rainier from Mount Pleasant',
 'dest': 'cyclistNerd',
 'was_comment': True,
 'body_html': '<!-- SC_OFF --><div class="md"><p>Really great feedback, I&#39;ll try booping the snoot. #helpful</p>\n</div><!-- SC_ON -->',
 'name': 't1_hygqut6',
 'created': 1645842416.0,
 'created_utc': 1645842416.0,
 'context': '/r/phcritiquetest/comments/t1lhzt/mount_rainier_from_mount_pleasant/hy

In [24]:
OPTOUT_PATTERN.search('I would like to optout plase')

<re.Match object; span=(16, 22), match='optout'>

In [25]:
re.search(r'\((\w*)\)', 're: your comment reply (abce234c)')

<re.Match object; span=(23, 33), match='(abce234c)'>

In [26]:
'CommunityGovProject' not in OPTED_OUT_USERS

True